<a href="https://colab.research.google.com/github/pgosar/AlphaHacks/blob/main/brand_embeddings/wikidata.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install fire
!pip install wikipedia

!pip install beautifulsoup4
!pip install google
!pip install requests

In [ ]:
import io
import os
import shutil
import re
import string
import tensorflow as tf
import numpy as np

import logging
import wikipedia
import random

from bs4 import BeautifulSoup as bs
import requests
import json

import nltk
from nltk.corpus import stopwords

from googlesearch import search

from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

In [ ]:
title_api = "https://en.wikipedia.org/w/api.php?action=query&format=json&titles="
search_api = "https://en.wikipedia.org/w/api.php?action=opensearch&limit=1&namespace=0&format=json&search="
pageid_api = "https://en.wikipedia.org/w/api.php?action=query&prop=extracts|revisions&rvslots=*&rvprop=content&format=json&pageids="
summary_api = "https://en.wikipedia.org/w/api.php?action=query&prop=extracts&exsentences=10&exlimit=1&explaintext=1&format=json&pageids="
summary_api_title = "https://en.wikipedia.org/w/api.php?action=query&prop=extracts&exsentences=10&exlimit=1&explaintext=1&format=json&titles="

In [ ]:
#GET LIST OF PAGE ID's
company_list_link = "https://en.wikipedia.org/wiki/List_of_food_companies"

res = requests.get(company_list_link)
soup = bs(res.text, "html.parser")
link_list = {}
page_dict = {}
for link in soup.find_all("a"):
    url = link.get("href", "")
    if "/wiki/" in url:
      link_list[link.text.strip()] = url

print(link_list)

for link_title in link_list:
  api_request = title_api + link_title
  response = requests.get(api_request)
  json_data = response.json()
  try:
    temp_value = list((json_data['query']['pages'].keys()))[0]
    print(temp_value)
    page_dict[link_title] = temp_value
  except KeyError:
    print("KeyError w/ ", link_title)
    continue

print(page_dict)

In [ ]:
with open('init_page_dict.json', 'w') as fp:
    json.dump(page_dict, fp)

In [ ]:
with open('init_page_dict.json') as fp:
  page_dict = json.load(fp)

In [ ]:
search_dict = {title: page_dict[title] for title in list(page_dict) if page_dict[title] == '-1'}

In [ ]:
keyword_list = ['brand', 'company', 'business', 'retail', 'manufact']
popped_list =[]

for title, id in search_dict.copy().items():
  found_alt = False
  for j in search(title + "food company", tld="co.in", num=5, stop=10, pause=2):
    if "wikipedia" in j:
      #print(j)
      new_name = j.split('/')[-1]
      #print(new_name)
      found_alt = True
      break

  search_dict.pop(title)
  if found_alt:
    result = requests.get(summary_api_title + new_name)
    json_data = result.json()
    new_id = list((json_data['query']['pages'].keys()))[0]
    print("New Name: {}, New ID: {}".format(new_name, new_id))
    search_dict[new_name] = new_id

  page_dict.pop(title)

In [ ]:
#Canon
for title, id in search_dict.copy().items():
  if "List" in title or id == '-1' or "wiki" in title.lower():
    popped_list.append(title)
    search_dict.pop(title)
    continue

In [ ]:
for title, id in search_dict.items():
  page_dict[title] = id